In [1]:
#used to import data
import pandas as pd
#Evaluates the model
from sklearn.metrics import log_loss
import time #just used to see how long the model takes to run
from pytorch_tabnet.tab_model import TabNetRegressor

from sklearn.model_selection import train_test_split

In [2]:
df_11 = pd.read_csv('data/pbp/ready_to_model/version_4/10_11_pbp_cleaned.csv')
df_12 = pd.read_csv('data/pbp/ready_to_model/version_4/11_12_pbp_cleaned.csv')
df_13 = pd.read_csv('data/pbp/ready_to_model/version_4/12_13_pbp_cleaned.csv')
df_14 = pd.read_csv('data/pbp/ready_to_model/version_4/13_14_pbp_cleaned.csv')
df_15 = pd.read_csv('data/pbp/ready_to_model/version_4/14_15_pbp_cleaned.csv')
df_16 = pd.read_csv('data/pbp/ready_to_model/version_4/15_16_pbp_cleaned.csv')
df_17 = pd.read_csv('data/pbp/ready_to_model/version_4/16_17_pbp_cleaned.csv')
df_18 = pd.read_csv('data/pbp/ready_to_model/version_4/17_18_pbp_cleaned.csv')
df_19 = pd.read_csv('data/pbp/ready_to_model/version_4/18_19_pbp_cleaned.csv')
df_20 = pd.read_csv('data/pbp/ready_to_model/version_4/19_20_pbp_cleaned.csv')
df_21 = pd.read_csv('data/pbp/ready_to_model/version_4/20_21_pbp_cleaned.csv')
df = pd.concat([df_11,df_12,df_13,df_14,df_15,df_16,df_17])

In [3]:
def get_X_y(df):
    ''' This function takes in a dataframe and splits it into the X and y variables
    '''
    X = df.drop(['is_goal'], axis=1).to_numpy()
    y = df.is_goal.to_numpy()
    
    return X,y

X,y = get_X_y(df)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.50, random_state=101)

#X_train.values
y_train = y_train.reshape(-1,1)
y_val = y_val.reshape(-1,1)
y_test = y_test.reshape(-1,1)

In [4]:
reg = TabNetRegressor(device_name = 'cuda')
model = reg.fit(X_train, y_train, 
                eval_set = [(X_train, y_train), (X_val, y_val)],
                eval_name=['train','val'],
                eval_metric = ['logloss'],
                max_epochs = 20)

Device used : cpu
epoch 0  | loss: 0.06328 | train_logloss: 0.21154 | val_logloss: 0.21529 |  0:00:33s
epoch 1  | loss: 0.052   | train_logloss: 0.20675 | val_logloss: 0.21182 |  0:01:07s
epoch 2  | loss: 0.05122 | train_logloss: 0.19965 | val_logloss: 0.2023  |  0:01:40s


C:\Users\Wolgy\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2279: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


epoch 3  | loss: 0.05081 | train_logloss: inf     | val_logloss: 0.20381 |  0:02:13s
epoch 4  | loss: 0.05058 | train_logloss: 0.19483 | val_logloss: 0.19961 |  0:02:46s
epoch 5  | loss: 0.05036 | train_logloss: 0.19367 | val_logloss: 0.19837 |  0:03:19s
epoch 6  | loss: 0.05027 | train_logloss: 0.19451 | val_logloss: 0.19916 |  0:03:52s
epoch 7  | loss: 0.05013 | train_logloss: 0.19393 | val_logloss: 0.19838 |  0:04:24s
epoch 8  | loss: 0.05006 | train_logloss: 0.19376 | val_logloss: 0.19815 |  0:04:57s
epoch 9  | loss: 0.05003 | train_logloss: 0.19373 | val_logloss: 0.19806 |  0:05:30s
epoch 10 | loss: 0.04994 | train_logloss: 0.19521 | val_logloss: 0.1994  |  0:06:03s
epoch 11 | loss: 0.04982 | train_logloss: 0.19229 | val_logloss: 0.19701 |  0:06:36s
epoch 12 | loss: 0.04975 | train_logloss: 0.19433 | val_logloss: 0.19801 |  0:07:09s
epoch 13 | loss: 0.04981 | train_logloss: 0.19334 | val_logloss: 0.19809 |  0:07:41s
epoch 14 | loss: 0.04972 | train_logloss: 0.19719 | val_logloss: 

In [5]:
preds = reg.predict(X_test)
print(log_loss(y_test, preds))

0.18829929074614263


In [ ]:
#after 10 epochs logloss = .197
#after 10 epochs logloss = .197
#Added a validation set, 10 epochs logloss = .198
#With the validation set and 10 epochs, logloss = .1965 <--- slight improvement
#increased the number of steps to 5, logloss = .200
#increased the number of steps to 8, logloss = .207
#increased the number of steps to 2, logloss = .1976
#using 3 as the default fir number of steps is fine 
#Setting the n_independent to 4 results in heavy overfitting
#set n_shared to 4 and logloss increased
#Setting labmda_sparse to 2.5e-4 was slightly better, logloss = .1965
#using entmax instead of sparsemax logloss = .1976
#Increasing the batch size did have a high logloss = .1985
#however, it was still falling, I will repeat with 20 epochs and compare
#increasing the epochs did little for the performance
#Increasing the num_workers to 2 gave a logloss of .1972 but was still falling, will allow more epochs
#After 25 epochs I got logloss = .1966
#in the end the defaults worked the best. 